In [6]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df_train = pd.read_pickle('../input/feats/df_trn_feat0.pkl')
df_test  = pd.read_pickle('../input/feats/df_test_feat0.pkl')

how to do stratified kfold in continuous variable

mean encode

- by categorical 

In [10]:
import gc
from sklearn.model_selection import StratifiedKFold

In [37]:
def mean_encode_train(df, cols, nfolds=4, num_bucket=100, target='deal_probability'):
    df_new = df.copy()
    
    
    ## create empty col
    for col in cols:
        df_new[col+'_mean_target'] = 0
    
    
    y_tr = df_new[target].values
    
    ## stratified k fold in continus y
    percentiles = np.percentile(y_tr, np.linspace(0, 100, num_bucket)) ## create percentiles 
    falls_into = np.searchsorted(percentiles,y_tr) ## fall into bucket
    
    skf = StratifiedKFold(nfolds, random_state=0)
    skf.get_n_splits(df_new, falls_into)
    
    for fold, (tr_idx, val_idx) in enumerate(skf.split(df_new, falls_into)):
        X_tr ,X_val = df_new.iloc[tr_idx].copy(), df_new.iloc[val_idx].copy()
        
        
        print('fold:{} mean encoding...'.format(fold),end='\n\t')
        
        for col in cols:        
            print(col,end='\t')
            means = X_val[col].map(X_tr.groupby(col)[target].mean()) ## map mean encoding in X_tr to X_val
            X_val[col + '_mean_target'] = means.astype('float16')

        df_new.iloc[val_idx] = X_val
        print('')    
        del X_tr,X_val;gc.collect()

    prior = df[target].mean()
    mean_target_cols = [col+'_mean_target' for col in cols]
    df_new[mean_target_cols] = df_new[mean_target_cols].fillna(prior)
    
    print('complete')
    return df_new[mean_target_cols]

In [38]:
def mean_encode_test(df_train, df_test, cols, num_bucket=100,target='deal_probability'):
    ''' encode df_test mean target by df_train'''
    y_target = df_train[target]
    
    for col in cols:
        print(col,end='\t')
        means = df_test[col].map(df_train.groupby(col)[target].mean()) ## map mean encode in train to test
        df_test[col+'_mean_target'] = means.astype('float16')
    print('')
    prior = df_train[target].mean()
    mean_target_cols = [col+'_mean_target' for col in cols]
    
    df_test[mean_target_cols] = df_test[mean_target_cols].fillna(prior)
    print('complete')
    return df_test[mean_target_cols]

In [39]:
df_train.columns

Index(['uidx', 'iidx', 'iid', 'region_city_label', 'tit_len', 'desc_len',
       'activation_date', 'month', 'day', 'weekday', 'param_1', 'param_2',
       'param_3', 'user_type', 'parent_category_name', 'price',
       'category_name', 'image_top_1', 'ads_cnt_by_uid', 'ads_cnt_by_iid',
       'deal_probability'],
      dtype='object')

In [43]:
encode_cols = [
    'uidx','iidx',
    'region_city_label',
    'month','day',
    'param_1','param_2','param_3',
    'user_type',
    'parent_category_name','category_name'
]
df_train_mean_target = mean_encode_train(df_train,cols=encode_cols)

fold:0 mean encoding...
	uidx	iidx	region_city_label	month	day	param_1	param_2	param_3	user_type	parent_category_name	category_name	
fold:1 mean encoding...
	uidx	iidx	region_city_label	month	day	param_1	param_2	param_3	user_type	parent_category_name	category_name	
fold:2 mean encoding...
	uidx	iidx	region_city_label	month	day	param_1	param_2	param_3	user_type	parent_category_name	category_name	
fold:3 mean encoding...
	uidx	iidx	region_city_label	month	day	param_1	param_2	param_3	user_type	parent_category_name	category_name	
complete


In [41]:
df_test_mean_target = mean_encode_test(df_train, df_test,cols=encode_cols)

uidx	iidx	region_city_label	month	day	param_1	param_2	param_3	user_type	parent_category_name	category_name	
complete


In [44]:
print('shape of df_train_mean_target:',df_train_mean_target.shape)
print('shape of df_test_mean_target:',df_test_mean_target.shape)

shape of df_train_mean_target: (1503424, 11)
shape of df_test_mean_target: (508438, 11)


In [45]:
store = pd.HDFStore('../input/feats/df_mean_target.h5') 
store['df_trn_mean_target'] = df_train_mean_target
store['df_test_mean_target'] = df_test_mean_target
store.close()